In [26]:
import pandas as pd
matches=pd.read_csv("matches.csv",index_col=4)
matches.shape
matches[matches["team"]=="Liverpool"]
matches["data"]=pd.to_datetime(matches["date"])
matches["venue_code"]=matches["venue"].astype("category").cat.codes
matches["opp_code"]=matches["opponent"].astype("category").cat.codes
matches["hour"]=matches["time"].str.replace(":.+","",regex=True).astype("int")
matches["date"]=pd.to_datetime(matches["date"])
matches["day_code"]=matches["date"].dt.dayofweek
matches["target"]=(matches["result"]=="W").astype("int")

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf=RandomForestClassifier(n_estimators=50,min_samples_split=10,random_state=1)
train=matches[matches["date"]<"2022-01-01"]
test=matches[matches["date"]>"2022-01-01"]
predictors=["venue_code","opp_code","hour","day_code"]
rf.fit(train[predictors],train["target"])
preds=rf.predict(test[predictors])
from sklearn.metrics import accuracy_score
acc=accuracy_score(test["target"],preds)
def roll(group,cols,new_cols):
    group=group.sort_values("date")
    roll_st=group[cols].rolling(3,closed='left').mean()
    group[new_cols]=roll_st
    group=group.dropna(subset=new_cols)
    return group
from sklearn.metrics import precision_score
cols=["gf","ga","sh","sot","dist","fk","pk","pkatt"]
new_cols=[f"{c}_rolling" for c in cols] 
matches_rolling=matches.groupby("team").apply(lambda x:roll(x,cols,new_cols))

matches_rolling=matches_rolling.droplevel("team")
matches_rolling.index=range(matches_rolling.shape[0])

def make_predictions(data, predictors):
    train = data[data["date"] < '2022-01-01']
    test = data[data["date"] > '2022-01-01']
    rf.fit(train[predictors], train["target"])
    preds = rf.predict(test[predictors])
    combined = pd.DataFrame(dict(actual=test["target"], predicted=preds), index=test.index)
    error = precision_score(test["target"], preds)
    return combined, error
combined,error=make_predictions(matches_rolling,predictors+new_cols)
print(error)
combined=combined.merge(matches_rolling[["date","team","opponent","result"]],left_index=True,right_index=True)
combined
class MissingDict(dict):
    __missing__=lambda self,key:key
map_values = {"Brighton and Hove Albion": "Brighton", "Manchester United": "Manchester Utd", "Newcastle United": "Newcastle Utd", "Tottenham Hotspur":"Tottenham","West Ham United":"West Ham","Woverhampton Wanderrers":"Wolves"}
mapping=MissingDict(**map_values)    
combined["new_team"]=combined["team"].map(map_values)
combined
merged=combined.merge(combined,left_on=["date","new_team"],right_on=["date","opponent"])
merged[(merged["predicted_x"] == 1) & (merged["predicted_y"] ==0)]["actual_x"].value_counts()

0.625


1    11
0     5
Name: actual_x, dtype: int64